<a href="https://colab.research.google.com/github/vsidaarth/Transformer-Variants-for-HourvAhead-PV-Forecasting/blob/main/Informer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Add Informer repo to sys.path
import sys
sys.path.append('/content/drive/MyDrive/Informer2020')


In [ ]:
# Load Informer using importlib (bypasses Python's package limitations on Drive)
import importlib.util

module_path = "/content/drive/MyDrive/Informer2020/models/model.py"
spec = importlib.util.spec_from_file_location("informer_model", module_path)
informer_module = importlib.util.module_from_spec(spec)
sys.modules["informer_model"] = informer_module
spec.loader.exec_module(informer_module)

# ✅ Informer class is now available
Informer = informer_module.Informer


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import plotly.express as px

In [ ]:
# load the data
df = pd.read_csv('/content/updated_timetable.txt')
df.columns=['datetime','irradiation_forecast','temperature_forecast','irradiation','temperature','power']
tdi = pd.to_datetime(df['datetime'], format='%d-%m-%y %H')
df.set_index(tdi, inplace=True)

# # Define date ranges to remove
# ranges_to_remove = [
#     ('2018-05-01', '2018-05-22'),
#     ('2019-09-09', '2019-09-28'),
#     ('2019-11-29', '2020-01-20'),
#     ('2013-07-23', '2013-08-26')
# ]

# # Create a boolean mask for rows to keep
# mask = pd.Series(True, index=df.index)
# for start, end in ranges_to_remove:
#     mask &= ~((df.index >= start) & (df.index <= end))

# # Apply the mask to the DataFrame
# data = df[mask]
start_date = '2014-01-01'
end_date = '2019-01-31'
# Filter the data for the specified date range
data = df[start_date:end_date]

data = data.drop(columns=['datetime','irradiation_forecast','temperature_forecast'])
data['power'] = data['power']/1000

# data['power'] = data['power']/data['power'].max()*317
# data = data[59905::]

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

# Ensure zero values are plotted correctly
# data_new.loc[data_new['power_generation'].isna(), 'power_generation'] = 0

fig = make_subplots(rows=1, cols=1)
# Plotting the Load Characteristic curve
fig.add_trace(
    go.Scatter(x=data.index, y=data['power'], name='Power Generation'),
    row=1, col=1
)

# Updating axis titles
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Power Generation (kW)", row=1, col=1)

# Updating the layout
fig.update_layout(
    height=600, width=1500, template='simple_white',
    title={
        'text': "Power Generation",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(size=14, color="Black"),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()


In [ ]:
load = data.loc[:, 'power']
scaler = MinMaxScaler()
load = scaler.fit_transform(load.values.reshape(-1, 1)).flatten()

irradiation = data.loc[:, 'irradiation']
scaler_w = MinMaxScaler()
irradiation = scaler_w.fit_transform(irradiation.values.reshape(-1, 1)).flatten()

load_scaled = pd.DataFrame({
    'power_generation': load,
    'irradiation': irradiation
}, index=data.index)


shift_1 = load_scaled.shift(1,axis = 0)
shift_1.columns = ['power_generation(t-1)', 'irradiation(t-1)']

final_table = pd.concat([load_scaled['power_generation'],shift_1],axis = 1)
final_table = final_table.dropna(axis=0)

# Extracting components
final_table['hour'] = final_table.index.hour
final_table['day_of_week'] = final_table.index.weekday
final_table['month'] = final_table.index.month

# Applying cyclical encoding
final_table['hour_sin'] = np.sin(2 * np.pi * final_table['hour'] / 24)
final_table['hour_cos'] = np.cos(2 * np.pi * final_table['hour'] / 24)
final_table['month_sin'] = np.sin(2 * np.pi * final_table['month'] / 12)
final_table['month_cos'] = np.cos(2 * np.pi * final_table['month'] / 12)

# Optionally drop the original time components if they are no longer needed
final_table.drop(['hour', 'day_of_week', 'month'], axis=1, inplace=True)

# Show the modified DataFrame
final_table.head(-1)


,power_generation,power_generation(t-1),irradiation(t-1),hour_sin,hour_cos,month_sin,month_cos
datetime,,,,,,,
2014-01-01 01:00:00,0.0,0.0,0.000441,0.258819,9.659258e-01,0.5,0.866025
2014-01-01 02:00:00,0.0,0.0,0.000336,0.500000,8.660254e-01,0.5,0.866025
2014-01-01 03:00:00,0.0,0.0,0.000228,0.707107,7.071068e-01,0.5,0.866025
2014-01-01 04:00:00,0.0,0.0,0.000186,0.866025,5.000000e-01,0.5,0.866025
2014-01-01 05:00:00,0.0,0.0,0.000017,0.965926,2.588190e-01,0.5,0.866025
...,...,...,...,...,...,...,...
2019-01-31 18:00:00,0.0,0.0,0.000892,-1.000000,-1.836970e-16,0.5,0.866025
2019-01-31 19:00:00,0.0,0.0,0.000920,-0.965926,2.588190e-01,0.5,0.866025
2019-01-31 20:00:00,0.0,0.0,0.001194,-0.866025,5.000000e-01,0.5,0.866025


In [ ]:
def split_sequences_autoformer(input_par, output, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(input_par)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out - 1
        if out_end_ix >= len(input_par):
            break
        seq_x = input_par[i:end_ix, :]
        seq_y = output[out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
test_set_start_date = final_table.index[-1] - pd.DateOffset(months=12)
train_set = final_table[final_table.index < test_set_start_date]
test_set = final_table[final_table.index >= test_set_start_date]

In [ ]:
# === Import Informer + Wrapper
# from informer_model import Informer
# import torch.nn as nn
NEW_SEQ_LEN  = 48
NEW_LABEL_LEN = 24
n_features = 6

n_steps_in = NEW_SEQ_LEN # new seq_len
n_steps_out = 1  # forecast horizon
input_sp = train_set.iloc[:,1:7]
output_sp = pd.DataFrame(train_set.loc[:,'power_generation'])
# Split sequences again
X_auto, y_auto = split_sequences_autoformer(input_sp.values, output_sp.values, n_steps_in, n_steps_out)

# Train-test split as before
input_train = train_set.iloc[:, 1:8]
output_train = pd.DataFrame(train_set['power_generation'])
input_test = test_set.iloc[:, 1:8]
output_test = pd.DataFrame(test_set['power_generation'])

X_train_auto, y_train_auto = split_sequences_autoformer(input_train.values, output_train.values, n_steps_in, n_steps_out)
X_test_auto, y_test_auto = split_sequences_autoformer(input_test.values, output_test.values, n_steps_in, n_steps_out)

# Convert again to torch tensors
X_train_tensor = torch.tensor(X_train_auto, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_auto, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_auto, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_auto, dtype=torch.float32)

class InformerWrapper(nn.Module):
    def __init__(self, config):
        super(InformerWrapper, self).__init__()
        self.model = Informer(
            enc_in=config.enc_in,
            dec_in=config.dec_in,
            c_out=config.c_out,
            seq_len=config.seq_len,
            label_len=config.label_len,
            out_len=config.pred_len,
            factor=config.factor,
            d_model=config.d_model,
            n_heads=config.n_heads,
            e_layers=config.e_layers,
            d_layers=config.d_layers,
            d_ff=config.d_ff,
            dropout=config.dropout,
            embed=config.embed,
            freq=config.freq,
            activation=config.activation,
            output_attention=config.output_attention,
            distil=config.distil,
            device=config.device
        )

    def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec):
        return self.model(x_enc, x_mark_enc, x_dec, x_mark_dec)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# === Config
class Configs:
    def __init__(self):
        self.seq_len = NEW_SEQ_LEN
        self.label_len = NEW_LABEL_LEN
        self.pred_len = 1
        self.enc_in = n_features
        self.dec_in = n_features
        self.c_out = 1
        self.factor = 3
        self.d_model = 64
        self.n_heads = 2
        self.e_layers = 1
        self.d_layers = 1
        self.d_ff = 256
        self.dropout = 0.05
        self.embed = 'fixed'
        self.freq = 'h'
        self.activation = 'gelu'
        self.output_attention = False
        self.distil = True
        self.device = device

configs = Configs()
model = InformerWrapper(configs).to(device)

with torch.no_grad():
    test_input = torch.rand((2, NEW_SEQ_LEN, n_features)).to(device)
    x_dec = torch.rand((2, NEW_LABEL_LEN + 1, n_features)).to(device)  # Match decoder mark
    x_mark_enc = torch.zeros((2, NEW_SEQ_LEN, 4)).to(device)
    x_mark_dec = torch.zeros((2, NEW_LABEL_LEN + 1, 4)).to(device)

    output = model(test_input, x_mark_enc, x_dec, x_mark_dec)
    print("Informer output shape:", output.shape)


# === Train
from torch.utils.data import Dataset, DataLoader

class LoadDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = LoadDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0

    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        x_enc = batch_X
        x_dec = torch.zeros((batch_X.shape[0], NEW_LABEL_LEN + 1, batch_X.shape[2])).to(device)
        x_mark_dec = torch.zeros((batch_X.shape[0], NEW_LABEL_LEN + 1, 4)).to(device)
        x_mark_enc = torch.zeros((batch_X.shape[0], NEW_SEQ_LEN, 4)).to(device)
        x_mark_dec = torch.zeros((batch_X.shape[0], NEW_LABEL_LEN + 1, 4)).to(device)

        optimizer.zero_grad()
        output = model(x_enc, x_mark_enc, x_dec, x_mark_dec)
        output = output[:, -1, 0]  # shape [B]
        loss = loss_fn(output, batch_y.view(-1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {epoch_loss/len(train_loader):.4f}")


# === Evaluate
model.eval()
predictions = []
true_values = []

with torch.no_grad():
    for i in range(0, len(X_test_tensor), 32):
        batch_X = X_test_tensor[i:i+32].to(device)

        x_enc = batch_X
        x_dec = x_dec = torch.zeros((batch_X.shape[0], NEW_LABEL_LEN + 1, batch_X.shape[2])).to(device)
        x_mark_enc = torch.zeros((batch_X.shape[0], NEW_SEQ_LEN, 4)).to(device)
        x_mark_dec = torch.zeros((batch_X.shape[0], NEW_LABEL_LEN + 1, 4)).to(device)

        output = model(x_enc, x_mark_enc, x_dec, x_mark_dec)
        output = output[:, -1, 0]  # shape [B]
        predictions.append(output.cpu().numpy())
        true_values.append(y_test_tensor[i:i+32].cpu().numpy())

predictions = np.concatenate(predictions, axis=0).reshape(-1, 1)
true_values = np.concatenate(true_values, axis=0).reshape(-1, 1)

# === Rescale
predictions_original = scaler.inverse_transform(
    np.concatenate([predictions.reshape(-1,1), np.zeros((len(predictions), 3))], axis=1))[:,0]
true_values_original = scaler.inverse_transform(
    np.concatenate([true_values.reshape(-1,1), np.zeros((len(true_values), 3))], axis=1))[:,0]

# === Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(true_values_original, predictions_original)
rmse = np.sqrt(mean_squared_error(true_values_original, predictions_original))
mbe = np.mean(predictions_original - true_values_original)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MBE: {mbe:.2f}")


Informer output shape: torch.Size([2, 1, 1])
Epoch [1/10], Loss: 0.0330
Epoch [2/10], Loss: 0.0166
Epoch [3/10], Loss: 0.0144
Epoch [4/10], Loss: 0.0130
Epoch [5/10], Loss: 0.0124
Epoch [6/10], Loss: 0.0121
Epoch [7/10], Loss: 0.0116
Epoch [8/10], Loss: 0.0115
Epoch [9/10], Loss: 0.0112
Epoch [10/10], Loss: 0.0109
MAE: 0.26
RMSE: 0.46
MBE: 0.10


In [ ]:
import os
import pandas as pd
import numpy as np

# === Function to save forecasts ===
def save_forecasts(true_values_original, predictions_original, test_set, model_name, filename):
    # Ensure save folder exists in Google Drive
    save_dir = "/content/drive/MyDrive/Colab Notebooks/Forecasts"
    os.makedirs(save_dir, exist_ok=True)

    # Build full output path
    out_path = os.path.join(save_dir, filename)

    # Align lengths
    n = min(len(true_values_original), len(predictions_original), len(test_set.index))

    # Build dataframe
    df = pd.DataFrame({
        "datetime": pd.to_datetime(test_set.index[:n]),
        "actual": np.asarray(true_values_original).reshape(-1)[:n],
        "forecast": np.asarray(predictions_original).reshape(-1)[:n],
        "model": model_name
    })

    # Save to CSV
    df.to_csv(out_path, index=False)
    print(f"[{model_name}] CSV saved to: {out_path}")
    return df.head()

# === Example: Save Informer results ===
save_forecasts(
    true_values_original,      # your ground truth (kW)
    predictions_original,      # your model forecasts (kW)
    test_set,                  # DataFrame with datetime index
    "Informer",                # model name
    "preds_informer.csv"       # file name in Drive
)


[Informer] CSV saved to: /content/drive/MyDrive/Colab Notebooks/Forecasts/preds_informer.csv


,datetime,actual,forecast,model
0,2018-01-31 23:00:00,0.0,0.074689,Informer
1,2018-02-01 00:00:00,0.0,0.075693,Informer
2,2018-02-01 01:00:00,0.0,0.076901,Informer
3,2018-02-01 02:00:00,0.0,0.077332,Informer
4,2018-02-01 03:00:00,0.0,0.077038,Informer
